In [ ]:
import os
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"
import datasets, transformers, onnx, onnxruntime, torch, torch.nn as nn, numpy as np
from transformers import AutoTokenizer, AutoModelForSequenceClassification

using BERT-base-uncased: https://huggingface.co/google-bert/bert-base-uncased

In [ ]:
checkpoint = "google-bert/bert-base-uncased"

In [ ]:
dataset = datasets.load_dataset("emotion")
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True, max_length=512)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
full_train_dataset = tokenized_datasets["train"]
full_eval_dataset = tokenized_datasets["test"]

In [ ]:
print(len(full_train_dataset))
print(len(full_eval_dataset))

In [ ]:
device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=6)
model = model.to(device)

In [ ]:
metric = datasets.load_metric("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import TrainingArguments, Trainer
training_args = TrainingArguments("test_trainer",
                                  per_device_train_batch_size=32,
                                  gradient_accumulation_steps=4,
                                  num_train_epochs=24, learning_rate=3e-05,
                                  evaluation_strategy="epoch",
                                  fp16=True)
trainer = Trainer(
    model = model,
    args=training_args,
    train_dataset=full_train_dataset,
    eval_dataset=full_eval_dataset
)

In [10]:
trainer.train()